# Space Invaders

## Dependencies

In [1]:
!pip install tensorflow==2.9.1 gym keras-rl2 gym[atari]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ROM instructions: https://github.com/openai/atari-py#roms

In [2]:
!python -m atari_py.import_roms roms

copying space_invaders.bin from roms/Space Invaders.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/space_invaders.bin


## Exploration and baseline

In [4]:
import gym
import random
import numpy as np

In [5]:
env = gym.make("SpaceInvaders-v4")
print(env.observation_space.shape)

(210, 160, 3)


In [6]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [7]:
EPISODES = 100
scores = []
for episode in range(1, EPISODES + 1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        # env.render()
        action = random.choice(range(env.action_space.n))
        n_state, reward, done, info = env.step(action)
        score += reward
    
    scores.append(score)
    print(f"Episode {episode}: Reward == {score}")

avg = np.mean(scores)
print(f"Average reward: {avg}")
env.close()

Episode 1: Reward == 180.0
Episode 2: Reward == 290.0
Episode 3: Reward == 55.0
Episode 4: Reward == 225.0
Episode 5: Reward == 65.0
Episode 6: Reward == 65.0
Episode 7: Reward == 150.0
Episode 8: Reward == 155.0
Episode 9: Reward == 10.0
Episode 10: Reward == 60.0
Episode 11: Reward == 55.0
Episode 12: Reward == 165.0
Episode 13: Reward == 225.0
Episode 14: Reward == 190.0
Episode 15: Reward == 45.0
Episode 16: Reward == 40.0
Episode 17: Reward == 195.0
Episode 18: Reward == 240.0
Episode 19: Reward == 135.0
Episode 20: Reward == 60.0
Episode 21: Reward == 210.0
Episode 22: Reward == 155.0
Episode 23: Reward == 75.0
Episode 24: Reward == 170.0
Episode 25: Reward == 75.0
Episode 26: Reward == 150.0
Episode 27: Reward == 155.0
Episode 28: Reward == 110.0
Episode 29: Reward == 55.0
Episode 30: Reward == 70.0
Episode 31: Reward == 110.0
Episode 32: Reward == 280.0
Episode 33: Reward == 190.0
Episode 34: Reward == 55.0
Episode 35: Reward == 335.0
Episode 36: Reward == 100.0
Episode 37: Rew

So the baseline is around 150.

## Model

In [8]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D, Resizing, Rescaling, Reshape
from tensorflow.python.keras.layers.core import Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.image import rgb_to_grayscale
from tensorflow.keras.layers import Layer
from tensorflow.keras.utils import register_keras_serializable

In [9]:
@register_keras_serializable("atari")
class GrayscaleLayer(Layer):
  def call(self, input):
    return rgb_to_grayscale(input)


In [10]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Input(shape=(height, width, channels)))
    model.add(GrayscaleLayer())
    model.add(Resizing(height // 2, width // 2))
    model.add(Rescaling(1./255)) # normalize to [0, 1]
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu'))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    # model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [11]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [12]:
model = build_model(height, width, channels, actions)

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 grayscale_layer (GrayscaleL  (None, 210, 160, 1)      0         
 ayer)                                                           
                                                                 
 resizing (Resizing)         (None, 105, 80, 1)        0         
                                                                 
 rescaling (Rescaling)       (None, 105, 80, 1)        0         
                                                                 
 conv2d (Conv2D)             (None, 25, 19, 32)        2080      
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 8, 64)         32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 6, 64)          36928     
                                                        

## Agent

In [14]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [15]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(
        EpsGreedyQPolicy(), 
        attr='eps', 
        value_max=1.0, 
        value_min=0.1, 
        value_test=0.2, 
        nb_steps=10000
    )
    memory = SequentialMemory(
        limit=1000, 
        window_length=4
    )
    dqn = DQNAgent(
        model=model, 
        memory=memory, 
        policy=policy,
        enable_dueling_network=True, 
        dueling_type='avg', 
        nb_actions=actions, 
        nb_steps_warmup=1000
    )
    return dqn

In [16]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

RecursionError: ignored

## Train

In [92]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...


ValueError: ignored

In [ ]:
scores = dqn.test(env, nb_episodes=20, visualize=False)
np.mean(scores.history["episode_reward"])